In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
def read_ubyte_file(file_path):
    with open(file_path, 'rb') as f:
        # Read the magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        
        if magic_number == 2051:  # Magic number for images
            num_images = int.from_bytes(f.read(4), 'big')
            rows = int.from_bytes(f.read(4), 'big')
            cols = int.from_bytes(f.read(4), 'big')
            images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows, cols)
            return images
        
        elif magic_number == 2049:  # Magic number for labels
            num_labels = int.from_bytes(f.read(4), 'big')
            labels = np.frombuffer(f.read(), dtype=np.uint8)
            return labels
        
        else:
            raise ValueError('Invalid magic number in file header')

train_images_path = './data/train-images-idx3-ubyte'
train_labels_path = './data/train-labels-idx1-ubyte'

train_images = read_ubyte_file(train_images_path)
train_labels = read_ubyte_file(train_labels_path)


In [7]:
print(train_images.shape)
# go from (60000, 28, 28) to (60000, 784)
train_images = train_images.reshape(train_images.shape[0],train_images.shape[1]*train_images.shape[2])
print(train_images.shape)

(60000, 28, 28)
(60000, 784)
